In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import *
from nltk import word_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pickle
import string

C:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
!pip install tensorflow == 2.5.0

ERROR: Invalid requirement: '=='


In [3]:
!pip install keras

In [4]:
!pip install konlpy

In [5]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to c:\users\jsy\appdata\local\temp\pip-req-build-7adxodrg


  Running command git clone -q https://github.com/ssut/py-hanspell.git 'C:\Users\jsy\AppData\Local\Temp\pip-req-build-7adxodrg'


In [7]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\jsy\appdata\local\temp\pip-req-build-mkn97d8i
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\jsy\AppData\Local\Temp\pip-req-build-mkn97d8i'


In [6]:
from pykospacing import Spacing
from hanspell import spell_checker
import tensorflow as tf

In [12]:
# 총 56253행...
tc = pd.read_csv('tc_v1.csv')

In [13]:
tc.drop(columns = ["Unnamed: 0"], inplace=True)
tc.columns = ['title', 'contents']

df = pd.DataFrame()
df["text"] = tc["title"].map(str) + " " + tc["contents"]

In [14]:
# 중복 행 제거 
df = df.drop_duplicates(['text'])
df.reset_index(drop = True,inplace = True)

# 결측치 제거
df = df.dropna(axis = 0)
len(df) #55818개

55818

In [21]:
# 전처리 코드 
def clean_text(text):
    text = re.sub('http[s]?://\S+', '', text) # http url 제거
    text = re.sub('\S*@\S*\s?', '', text) #
    text = re.sub(r'\[.*?\]', '', text) # 
    text = re.sub(r'\w*\d\w*', '', text) # 숫자 포함하는 텍스트 제거
    text = re.sub('[?.,;:|\)*~"`’!\-_+<>@\#$%&-=#}※]', ' ', text)#특수문자 이모티콘 제거.
    text = re.sub("\n", '', text) #개행문자 제거
    text = re.sub("\r", '', text) #개행문자 제거
    text = re.sub("\xa0", '', text) #개행문자 제거
    text = re.sub(r'이전 게시물', '', text)
    text = re.sub(r'Copyright .* rights reserved', '', text) # "Copyright all rights reserved" 제거
    text = re.sub(r'dc official App', '', text) # "dc official App" 제거
    return text

In [22]:
df['text_clean']=pd.DataFrame(df.text.apply(lambda x: clean_text(x)))

In [23]:
spacing = Spacing()

'형들 말 참고해서 씨앗 벗겨 옴 오렌지 먹다가 나온 씨앗이 하나는 씨앗 벗기다 힘 조절 실패로 부숴버림ㅠ'

In [24]:
## 띄어쓰기
import time
import sys

start = time.time() 
df['text_sep']  = df['text_clean'].apply(lambda x: spacing(x))    
print("time :", time.time() - start) #초

time : 2126.717365026474


In [27]:
## 맞춤법
def spell(docu):
    try:
        docu_spelled= spell_checker.check(docu).checked
    except:
        docu_spelled = docu
    return docu_spelled

start = time.time()  
df['text_p'] = df['text_sep'].apply(lambda x: spell(x))
print("time :", time.time() - start) #초

time : 4780.813404083252


In [28]:
# 중간 저장
df.to_csv('tc_v2.csv',encoding='utf-8')